# Decorator Exercise Solutions

Complete solutions with explanations.

---

## Solution 1: Timing Decorator

In [ ]:
import time
from functools import wraps

def timer(func):
    """Decorator that times function execution."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = (time.time() - start) * 1000  # Convert to ms
        print(f"⏱️ {func.__name__} took {elapsed:.2f}ms")
        return result
    return wrapper

# Test
@timer
def slow_function():
    """Simulates slow operation."""
    time.sleep(0.1)
    return "Done!"

result = slow_function()
assert result == "Done!"
assert slow_function.__name__ == "slow_function"
print("✅ Test passed!")

## Solution 2: Retry Decorator

In [ ]:
from functools import wraps

def retry(max_attempts: int):
    """Decorator factory that retries function on exception."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(1, max_attempts + 1):
                try:
                    result = func(*args, **kwargs)
                    print(f"Attempt {attempt} succeeded!")
                    return result
                except Exception as e:
                    last_exception = e
                    print(f"Attempt {attempt} failed: {e}")
                    if attempt == max_attempts:
                        raise
            
            raise last_exception
        
        return wrapper
    return decorator

# Test
attempt_count = 0

@retry(max_attempts=3)
def unreliable_function():
    """Fails twice, then succeeds."""
    global attempt_count
    attempt_count += 1
    if attempt_count < 3:
        raise ConnectionError("Connection error")
    return "Success!"

result = unreliable_function()
assert result == "Success!"
print("✅ Test passed!")

## Solution 3: Caching Decorator

In [ ]:
from functools import wraps

def cache(func):
    """Decorator that caches function results."""
    cached_results = {}
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Create cache key from arguments
        cache_key = (args, tuple(sorted(kwargs.items())))
        
        if cache_key in cached_results:
            print("Cache hit!")
            return cached_results[cache_key]
        
        # Compute and cache
        result = func(*args, **kwargs)
        cached_results[cache_key] = result
        return result
    
    return wrapper

# Test
@cache
def expensive_computation(n: int) -> int:
    """Simulates expensive computation."""
    print(f"Computing expensive_computation({n})")
    return n * n

result1 = expensive_computation(5)
result2 = expensive_computation(5)
result3 = expensive_computation(10)

assert result1 == 25
assert result2 == 25
assert result3 == 100
print("✅ Test passed!")

## Solution 4: Validation Decorator

In [ ]:
from functools import wraps
import inspect

def validate_types(func):
    """Decorator that validates function arguments match type hints."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Get function signature
        sig = inspect.signature(func)
        bound = sig.bind(*args, **kwargs)
        bound.apply_defaults()
        
        # Validate each argument
        for param_name, param_value in bound.arguments.items():
            param = sig.parameters[param_name]
            
            # Skip if no type hint
            if param.annotation == inspect.Parameter.empty:
                continue
            
            # Validate type
            expected_type = param.annotation
            if not isinstance(param_value, expected_type):
                raise TypeError(
                    f"Argument '{param_name}' must be {expected_type.__name__}, "
                    f"got {type(param_value).__name__}"
                )
        
        return func(*args, **kwargs)
    
    return wrapper

# Test
@validate_types
def greet(name: str, age: int) -> str:
    """Greet someone with their age."""
    return f"{name} is {age} years old"

# Valid call
result = greet("Alice", 30)
assert result == "Alice is 30 years old"

# Invalid call
try:
    greet("Bob", "thirty")
    assert False, "Should have raised TypeError"
except TypeError as e:
    print(f"✅ Caught type error: {e}")

print("✅ All tests passed!")

## Solution 5: Registry with Metadata

In [ ]:
from typing import Callable, Dict, Any, Optional
import inspect

class TransformRegistry:
    """Registry for transformations with metadata."""
    
    def __init__(self):
        self._transforms: Dict[str, Callable] = {}
        self._metadata: Dict[str, Dict[str, Any]] = {}
    
    def register(self, name: str, category: Optional[str] = None):
        """Register transformation with metadata."""
        def decorator(func: Callable) -> Callable:
            # Validate docstring
            if not func.__doc__ or len(func.__doc__.strip()) < 10:
                raise ValueError(
                    f"Transform '{name}' must have docstring (min 10 chars)"
                )
            
            # Validate uniqueness
            if name in self._transforms:
                raise ValueError(f"Transform '{name}' already registered")
            
            # Store
            self._transforms[name] = func
            self._metadata[name] = {
                'category': category,
                'docstring': func.__doc__.strip(),
                'signature': str(inspect.signature(func))
            }
            
            return func
        
        return decorator
    
    def get(self, name: str) -> Callable:
        """Get transformation by name."""
        if name not in self._transforms:
            raise ValueError(f"Transform '{name}' not found")
        return self._transforms[name]
    
    def list_by_category(self, category: str) -> Dict[str, Callable]:
        """List transformations by category."""
        return {
            name: func
            for name, func in self._transforms.items()
            if self._metadata[name]['category'] == category
        }
    
    def get_info(self, name: str) -> Dict[str, Any]:
        """Get full info for transformation."""
        return {
            'name': name,
            'function': self._transforms[name],
            **self._metadata[name]
        }

# Test
registry = TransformRegistry()

@registry.register('filter_nulls', category='filtering')
def filter_nulls(df, column: str):
    """Remove null values from specified column."""
    return df[df[column].notna()]

@registry.register('group_sum', category='aggregation')
def group_sum(df, group_by: str):
    """Sum values grouped by column."""
    return df.groupby(group_by).sum()

# Validate
assert len(registry.list_by_category('filtering')) == 1
assert len(registry.list_by_category('aggregation')) == 1

info = registry.get_info('filter_nulls')
assert info['category'] == 'filtering'
assert 'Remove null values' in info['docstring']

print("✅ All tests passed!")
print("\nRegistered transformations:")
for name in registry._transforms:
    info = registry.get_info(name)
    print(f"  - {name} [{info['category']}]: {info['docstring']}")

## Solution 6: Explanation Pattern (Advanced)

In [ ]:
from typing import Callable, Optional
from functools import wraps

class ExplanationWrapper:
    """Wrapper that adds .explain capability to functions."""
    
    def __init__(self, func: Callable):
        self.func = func
        self._explain_func: Optional[Callable] = None
        
        # Preserve metadata
        self.__name__ = func.__name__
        self.__doc__ = func.__doc__
        self.__dict__.update(func.__dict__)
    
    def __call__(self, *args, **kwargs):
        """Make the wrapper callable."""
        return self.func(*args, **kwargs)
    
    def explain(self, explain_func: Callable) -> Callable:
        """Register explanation function."""
        self._explain_func = explain_func
        return self.func  # Return original for chaining
    
    def get_explanation(self, **kwargs) -> str:
        """Get explanation with context."""
        if self._explain_func is None:
            raise ValueError(
                f"No explanation registered for '{self.func.__name__}'"
            )
        return self._explain_func(**kwargs)

def transformation(name: str):
    """Decorator to create transformations with explanation capability."""
    def decorator(func: Callable) -> ExplanationWrapper:
        return ExplanationWrapper(func)
    return decorator

# Test
@transformation('filter_threshold')
def filter_threshold(df, threshold):
    """Filter records above threshold."""
    return df[df['value'] > threshold]

@filter_threshold.explain
def explain_filter(threshold, **context):
    plant = context.get('plant', 'Unknown')
    return f"Filter {plant} records above {threshold}"

# Test callable
import pandas as pd
df = pd.DataFrame({'value': [1, 2, 3, 4, 5]})
result = filter_threshold(df, 3)
assert len(result) == 2

# Test explanation
explanation = filter_threshold.get_explanation(threshold=10, plant='Indianapolis')
assert explanation == "Filter Indianapolis records above 10"

print("✅ All tests passed!")
print(f"Explanation: {explanation}")

---

## 🎯 Key Takeaways

1. **@wraps is essential** - Always preserve function metadata
2. **Decorator factories** - Use nested functions for arguments
3. **State in decorators** - Use classes or closures
4. **Validation early** - Fail fast with clear error messages
5. **Registry pattern** - Decorators can register functions globally
6. **Explanation pattern** - Classes can add methods to wrapped functions

These patterns power Odibi's transformation system!